<a href="https://colab.research.google.com/github/JoaoOliv-A/GOES-16-Media-SST/blob/main/UltimoCodSST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
################# Bibliotecas importadas para execução do cod ######################

!pip install xarray
!pip install goes2go
!pip install rioxarray
!pip install geopandas
!pip install matplotlib
!pip install pandas==2.2.2
!pip install numpy
!pip install netCDF4
!pip install dask

In [ ]:
################# Download do arquivos nc por periodo ######################

#  -> nuvem pública da Amazon Web Services (AWS)

from goes2go import GOES

from datetime import datetime, timedelta

# Defina o intervalo de datas
G = GOES(satellite=16, product="ABI-L2-SSTF", domain='C')

start_date = datetime(2024, 2, 1)
end_date = datetime(2024, 2, 29)

for day in range((end_date - start_date).days + 1):
    current_date = start_date + timedelta(days=day)
    start_time_str = current_date.strftime('%Y-%m-%d 00:00')
    end_time_str = current_date.strftime('%Y-%m-%d 23:00')

    files = G.timerange(start=start_time_str, end=end_time_str)

    # Mostra todos os arquivos disponíveis para o dia atual
    for file in files:
        print(file)

In [ ]:

################# Codigo para calculo, reprojeção e novo nc ######################


import gc
import rioxarray as rxr
import numpy as np
import pandas as pd
import os
import xarray as xr
from glob import glob
from pathlib import Path
import matplotlib.pyplot as plt

epsg = 4674

# Defina o caminho base onde os arquivos estão localizados
base_path = '/root/data/noaa-goes16/ABI-L2-SSTF'
# Use glob para buscar todos os arquivos .nc em subdiretórios
pattern = os.path.join(base_path, '**', '*.nc')
arqs = glob(pattern, recursive=True)

# Função para salvar a média calculada em um arquivo NetCDF
def salvar_media(mes_ano, da_media_mensal, caminho_arquivo):
    da_media_mensal.to_netcdf(caminho_arquivo)

# Função para plotar e salvar a média
def plotar_media(da_media_mensal, mes_ano):
    plt.figure(figsize=(10, 7))
    da_media_mensal.plot(cmap='jet')
    plt.title(f'Média de Temperatura em °C - {mes_ano}')
    plt.grid(True)
    plt.savefig(f'MediaMensal_{mes_ano}.png')

# Dicionário para armazenar dados mensais
dados_mensais = {}

# Processamento dos arquivos
for arq in arqs:
    arq = Path(arq)
    strTempo = arq.name.split('_')[3][1:]
    data = pd.to_datetime(strTempo, format='%Y%j%H%M%S%f')
    mes_ano = data.strftime('%d-%m')

    # Abrir o arquivo NetCDF
    dsTemp = rxr.open_rasterio(arq, chunks={'x': 600, 'y': 600})
    daTempKelvin = dsTemp['SST'][0]

    # Filtrar valores de preenchimento
    daTempKelvin = daTempKelvin.where(daTempKelvin != 65535)

    # Aplicar o scale_factor e o add_offset
    scale_factor = daTempKelvin.attrs.get('scale_factor', 1.0)
    add_offset = daTempKelvin.attrs.get('add_offset', 0.0)

    # Calcular a temperatura em Kelvin
    daTempKelvin = (daTempKelvin * scale_factor) + add_offset

    # Converter para Celsius e filtrar valores anômalos
    daTempCelsius = daTempKelvin - 273.15
    daTempCelsius = daTempCelsius.where((daTempCelsius > -30) & (daTempCelsius < 50))

    # Armazenar os dados na lista do mês correspondente
    if mes_ano not in dados_mensais:
        dados_mensais[mes_ano] = []
    dados_mensais[mes_ano].append(daTempCelsius)

    # Limpar a memória após cada iteração
    rec = dsTemp['SST'][0]
    del dsTemp, daTempKelvin, daTempCelsius
    gc.collect()


# Calcular a média para cada mês, fazer a reprojeção, e salvar
for mes_ano, lista_dados in dados_mensais.items():
    # Alinhar dimensões dos dados e calcular a média ignorando NaNs
    stack_dados = np.stack([da.values for da in lista_dados], axis=0)
    media_mensal = np.nanmean(stack_dados, axis=0)

    # Criar um novo Dataset com a média mensal e a variável SST
    ds_media_mensal = xr.Dataset(
    {'SST': ([ 'y','x'], media_mensal)},
    coords={'y': lista_dados[0].coords['y'], 'x': lista_dados[0].coords['x']},
    attrs={'units': '°C', 'scale_factor': 1.0, 'add_offset': 0.0, 'EPSG': epsg}
    )

    # Reprojetar a média mensal antes de plotar
    ds_media_mensal = ds_media_mensal.rio.write_crs(rec.rio.crs)
    ds_media_mensal_reprojetado =  ds_media_mensal.rio.reproject(dst_crs=f'EPSG:{epsg}')

    # Salvar a média em um arquivo NetCDF
    caminho_arquivo = f'/root/data/noaa-goes16/media_{mes_ano}.nc'
    salvar_media(mes_ano, ds_media_mensal_reprojetado, caminho_arquivo)


    # Limpar a memória após salvar e plotar
    del stack_dados, media_mensal, ds_media_mensal, ds_media_mensal_reprojetado
    gc.collect()

In [ ]:
################# Copiar para o drive ######################

!cp -r /root/data/noaa-goes16/ /content/drive/MyDrive